### Prepare Repo

In [1]:
# !git clone https://github.com/facebookresearch/av_hubert.git

# %cd av_hubert
# !git submodule init
# !git submodule update
# !pip install scipy
# !pip install sentencepiece
# !pip install python_speech_features
# !pip install scikit-video

# %cd fairseq
# !python -m pip install ./

In [2]:
%cd E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert

import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/fairseq")
from fairseq import checkpoint_utils, options, tasks, utils
import cv2
import tempfile
import torch
from transformers import Trainer, TrainingArguments
import utils as avhubert_utils
from argparse import Namespace
from IPython.display import HTML
import numpy as np
import sys
print(sys.version)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import wandb
from torch.utils.data import Dataset, Subset
import os
import datetime

E:\university\FYT\repos\multi_modal_ser\finetune_encoder\audio_video\av_hubert\avhubert


D:\anaconda\envs\py37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]


### Download Model

In [3]:
# !wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/vsr/base_vox_433h.pt -O E:/check_pts/avhubert.pt

### Build Model Pipeline

In [4]:
user_dir = "E:/university/FYT/repos/multi_modal_ser/finetune_encoder/audio_video/av_hubert/avhubert"
utils.import_user_module(Namespace(user_dir=user_dir))
ckpt_path = "E:/check_pts/avhubert.pt"
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])  
model = models[0]
if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")

Checkpoint: fine-tuned


### Load Dataset

In [5]:
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/avhubert_ds.pt")
outputs = model.extract_finetune(mmser_ds[:2])
print(outputs[0].shape)

torch.Size([2, 500, 768])


### Define the model

In [6]:
from avhubert_classifier import AVHUBERTClassifier

In [7]:
classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
classifier(**mmser_ds[:4])

{'logits': tensor([[-0.1157, -0.0678,  0.0320, -0.0467],
         [-0.1130, -0.0573,  0.0261, -0.0348],
         [-0.1206, -0.0576,  0.0448, -0.0368],
         [-0.1300, -0.0434,  0.0379, -0.0503]], grad_fn=<AddmmBackward0>)}

### Build Train Test DS

In [8]:
meta_df_ = mmser_ds.df_
sess_dict = mmser_ds.df_.groupby("session").groups
all_indices = set(mmser_ds.df_.index.tolist())

sess_ds = {}
for sess in sess_dict:
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])

In [9]:
def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

### Run Pipeline

In [10]:
import json
sess_id = list(sess_dict.keys())[7]

avhubert_classifier = AVHUBERTClassifier(model, 768, 256, mmser_ds.df_["emotion_id"].nunique())
for param in avhubert_classifier.encoder.parameters():
    param.requires_grad = False

wandb.init()
print(sess_id)
train_set, val_set, test_set = build_ds(sess_id)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brucehu. Use `wandb login --relogin` to force relogin


Ses04M
Train Samples: 3789
Val Samples: 1264
Test Samples: 478


In [14]:
output_dir=os.path.join("check_pts", "HUBERT", sess_id, datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=7e-4
training_args.num_train_epochs=50

In [15]:
from avhubert_trainer import CustomTrainer, compute_metrics

trainer = CustomTrainer(
    model=avhubert_classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [16]:
eval_result = trainer.evaluate()
test_result = trainer.predict(test_set).metrics

KeyboardInterrupt: 

In [ ]:
FREEZE_PROJ_PATH = "/home/freeze/{}/projector".format(sess_id)
FREEZE_CLAS_PATH = "/home/freeze/{}/classifier".format(sess_id)
os.makedirs(FREEZE_PROJ_PATH, exist_ok=True)
os.makedirs(FREEZE_CLAS_PATH, exist_ok=True)

FREEZE_PROJ = os.path.join(FREEZE_PROJ_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")
FREEZE_CLAS = os.path.join(FREEZE_CLAS_PATH, datetime.datetime.now().date().strftime(format="%Y-%m-%d")+".pt")

torch.save(avhubert_classifier.projector.state_dict(), FREEZE_PROJ)
torch.save(avhubert_classifier.classifier.state_dict(), FREEZE_CLAS)

model.projector.load_state_dict(torch.load(FREEZE_PROJ))
model.classifier.load_state_dict(torch.load(FREEZE_CLAS))

print(eval_result)
print(test_result)

json_test = json.dumps(test_result, indent=4)
json_eval = json.dumps(eval_result, indent=4)

# Writing to sample.json
with open("{}_eval.json".format(sess_id), "w") as outfile:
    outfile.write(json_eval)
with open("{}_test.json".format(sess_id), "w") as outfile:
    outfile.write(json_test)


In [21]:
meta_df_ = mmser_ds.df_
sess_dict = meta_df_.groupby("session").groups
all_indices = set(meta_df_.index.tolist())
sess_ds = {}
for sess in sess_dict:
#     sess_ds[sess+"_train"] = Subset(mmser_ds, 
#                                     indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    sess_ds[sess+"_train"] = Subset(mmser_ds, 
                                    indices=sess_dict[sess])
    
    
    
SESS_ID = list(sess_dict.keys())[0]
train_set, val_set, test_set = build_ds(SESS_ID)

Train Samples: 380
Val Samples: 127
Test Samples: 507


In [23]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set, batch_size=2)
# next(iter(train_loader))

In [24]:
output_dir=os.path.join("avhubert")

training_args = TrainingArguments(output_dir,report_to="wandb")
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=4
training_args.logging_steps = 3 # int(1000/training_args.per_device_train_batch_size)
training_args.eval_steps = 3 #int(1000/training_args.per_device_train_batch_size)
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=1e-3
training_args.num_train_epochs=50

trainer = CustomTrainer(
    model=classifier,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

D:\anaconda\envs\py37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brucehu. Use `wandb login --relogin` to force relogin


KeyboardInterrupt: 